In [ ]:
# clone codes
import os
os.chdir('/content/')

!git clone https://github.com/neuphonic/neutts-air
!apt-get install -y espeak
%pip install -r neutts-air/requirements.txt
%pip install llama-cpp-python onnxruntime
!apt-get install time ffmpeg
!/usr/bin/time -v



In [ ]:
# prepare benchmark
!git clone https://github.com/skiloop/tts-benchmarks
os.chdir('/content/tts-benchmarks/')
!sh install.sh
%pip install -r requirements.txt

In [ ]:
# test and prepare models
import os
os.chdir('/content/neutts-air/')

!python -m examples.onnx_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_codes samples/dave.pt \
  --ref_text samples/dave.txt \
  --backbone neuphonic/neutts-air-q4-gguf
!python -m examples.basic_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_audio ./samples/dave.wav \
  --ref_text ./samples/dave.txt \
  --backbone neuphonic/neutts-air-q4-gguf
!python -m examples.basic_example \
  --input_text "My name is Dave, and um, I'm from London" \
  --ref_audio ./samples/dave.wav \
  --ref_text ./samples/dave.txt
!python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --ref_codes samples/dave.pt
!

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [30]:
# config performance test
import json

config = {
  "description": "TTS性能测试",
  "iterations": 5,
  "tests": [
    {
      "name": "neutts",
      "command": "python -m examples.basic_example --input_text \"My name is Dave, and um, I'm from London\" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt"
    },
    {
      "name": "neutts-gguf",
      "command": "python -m examples.basic_example --input_text \"My name is Dave, and um, I'm from London\" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx-gguf",
      "command": "python -m examples.onnx_example --input_text \"My name is Dave, and um, I'm from London\" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx",
      "command": "python -m examples.onnx_example --input_text \"My name is Dave, and um, I'm from London\" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt"
    },
    {
      "name": "edge-tts",
      "command": "edge-tts --text \"My name is Dave, and um, I'm from London\" --write-media ./outputs/dave-edge-tts.wav"
    },
  ]
}

with open('perftest.json', 'w') as f:
  json.dump(config, f)


In [31]:
# do performance test
!perftest -f perftest.json -o pefromance.json


性能测试工具
总共 4 个测试，每个运行 5 次


[1/4]
测试 [neutts]: python -m examples.basic_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt
  完成 5 次运行

[2/4]
测试 [neutts-gguf]: python -m examples.basic_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --backbone neuphonic/neutts-air-q4-gguf
  完成 5 次运行

[3/4]
测试 [neutts-onnx-gguf]: python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt --backbone neuphonic/neutts-air-q4-gguf
  完成 5 次运行

[4/4]
测试 [neutts-onnx]: python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt
  完成 5 次运行

性能测试摘要

名称                   实际时间            用户时间            系统时间            相对速度      
---------------------------------------------------------------------